In [1]:
import pandas as pd
import childespy
import numpy as np

import os
from os.path import join, exists
np.random.seed(0)

# Important: Run this cell only once per "restart runtime"
# for reproducibility of random seed.

In [2]:
from utils import split_gen

 ## General description of splits (this notebook addresses the "all" and "age" splits)
 
 **All split**
    
- The normal train and validation from the original query on CHILDES in "Generate data to fine-tune" notebook.
- Train, Val chosen 80/20 randomly using unique transcript IDs only.
- Doesn't have xxx or yyy.

**Age split**

- Split on <=36 months for young, and rest for old.
- Do all split process.
- Doesn't have xxx or yyy.

**Child split**
- Split on 6 children as found in Generate Phonological Analysis. See notebook "Generate child data for finetuning".
- Has 200 yyy for validation, randomly select from UNIQUE TRANSCRIPT IDS ONLY. The rest is train.
- Doesn't have xxx or yyy.

# Generate Train, Val for All and Age splits

In [3]:
#get all of the North American and British English adult and child utterances without xxx or yyy
#concatenate them at the the transcript level
#hold out 20% for validation 

In [9]:
corpora = childespy.get_sql_query('select * from corpus where \
collection_name in ("Eng-NA", "Eng-UK") and data_source = "CHILDES"')
corpora

R[write to console]: Using current database version: '2020.1'.



,id,name,collection_name,data_source,collection_id
1,32,Garvey,Eng-NA,CHILDES,2
2,33,Valian,Eng-NA,CHILDES,2
3,34,Bernstein,Eng-NA,CHILDES,2
4,35,Clark,Eng-NA,CHILDES,2
5,36,PetersonMcCabe,Eng-NA,CHILDES,2
...,...,...,...,...,...
57,221,Wells,Eng-UK,CHILDES,12
58,222,Gathburn,Eng-UK,CHILDES,12
59,223,Nuffield,Eng-UK,CHILDES,12
60,224,Lara,Eng-UK,CHILDES,12


In [10]:
childes_datasets = ",".join([str(x) for x in corpora.collection_id])

In [11]:
regenerate = True
if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, target_child_name, corpus_name, target_child_age, type from utterance where collection_name in ("Eng-NA", "Eng-UK") \
    and collection_id in ('+childes_datasets+') and speaker_code in ("MOT", "FAT","CHI")' , db_version = "2020.1")
    utt_glosses.to_csv('csv/utt_glosses.csv', index=False)
else: 
    utt_glosses = pd.read_csv('csv/utt_glosses.csv')

R[write to console]: Using supported database version: '2020.1'.



In [ ]:
utt_glosses = split_gen.drop_errors(utt_glosses) # Drop errors for age and all splits.

In [16]:
utt_glosses.head()

,gloss,transcript_id,id,utterance_order,speaker_code,target_child_name,corpus_name,target_child_age,type
1,now we need this,3261,279663,1,CHI,None,Garvey,NaN,trail off
2,we need we you don't,3261,279666,2,CHI,None,Garvey,NaN,declarative
3,need a pocketbook because I'm the mother,3261,279668,3,CHI,None,Garvey,NaN,declarative
4,I'm I'm the,3261,279670,5,CHI,None,Garvey,NaN,trail off
5,it won't work xxx,3261,279671,6,CHI,None,Garvey,NaN,declarative


## "all" split

In [4]:

base_path = 'data/new_splits'

all_utt_glosses = utt_glosses.copy() # Important because split_gen functions often mutate, not copy.
all_split_df, _, _ = split_gen.exec_split_gen(all_utt_glosses, 'all', 'all', base_dir = base_path)

# expectations for verbose output.
# 232 should be (4205071, 7) (because you added two extra fields)
# 233 likewise is (3959952, 7)

Beginning split gen call: all all


/home/nwong/chompsky/childes/child_listening_continuation/child-directed-listening/utils/split_gen.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gloss_with_punct'] = [x['speaker_code_simple'] + ' '+ x['gloss'].lower() + x['punct'] for x in data.to_dict('records')]


Files written to data/new_splits/all/all


## "age" split

In [6]:
young_df, old_df = split_gen.get_age_split_data(months = 36)

cleaned_young_split_df, young_tok, young_chi_tok = split_gen.exec_split_gen(young_df, 'age', 'young', base_dir = base_path)
cleaned_old_split_df, old_tok, old_chi_tok = split_gen.exec_split_gen(old_df, 'age', 'old', base_dir = base_path)

# The young and old need to be overwritten -- how?

Beginning split gen call: age young


/home/nwong/chompsky/childes/child_listening_continuation/child-directed-listening/utils/split_gen.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gloss_with_punct'] = [x['speaker_code_simple'] + ' '+ x['gloss'].lower() + x['punct'] for x in data.to_dict('records')]


Files written to data/new_splits/age/young
Beginning split gen call: age old
Files written to data/new_splits/age/old


## Verifications: Quick checks for consistency with the full notebook (on "all" split)

In [ ]:
# Note you refer to a partition as a phase to avoid confusion

In [56]:

# The fix gloss should now match?

import os
from os.path import join, exists

retrieve_path = split_gen.get_split_folder('all', 'all', base_path)
tok = pd.read_csv(join(retrieve_path, 'vocab.csv'))
chi_tok = pd.read_csv(join(retrieve_path, 'chi_vocab.csv'))


orig_tok = pd.read_csv('data/vocab.csv') # Saved by the original notebook in your run.
orig_chi_tok = pd.read_csv('data/chi_vocab.csv')


In [57]:

nan_filler = 'is_an_nan_need_to_fill'
orig_tok_sorted = orig_tok.sort_values('word').reset_index(drop = True).fillna(nan_filler)
tok_sorted = tok.sort_values('word').reset_index(drop = True).fillna(nan_filler)

orig_tok_sorted.equals(tok_sorted)

orig_tok_sorted.head()

assert all(orig_tok_sorted['word'] == tok_sorted['word'])
assert all(orig_tok_sorted['count'] == tok_sorted['count'])

# Note that if you compare the dataframes directly, like:
# print(orig_tok_sorted[orig_tok_sorted != tok_sorted])
# print(tok_sorted[orig_tok_sorted != tok_sorted])
# This is not going to work because of the Unnamed: 0 column. But the actual contents are the same.

In [58]:
# Checking if the dataframes are the same after the first cleaning

base_path = 'refactored'

my_out = pd.read_csv(join(base_path, 'cleaned_utt_glosses_my.csv'))
his_out = pd.read_csv('data/cleaned_utt_glosses_meylan.csv')

my_out.sort_values('id')
his_out.sort_values('id')

filler = 'fill_this_nan______'

my_out = my_out.fillna(filler)
his_out = his_out.fillna(filler)

for field in my_out.columns:
    if field not in his_out.columns: continue
    if not all(my_out[field] == his_out[field]):
        print(field)
        print(my_out[my_out[field] != his_out[field]])
        break
        
    # Seems like they are actually the same
    # So long as "NaN" glosses are filled with some value to be equivalent.